In [20]:
## Model Training

In [23]:
import pandas as pd
df = pd.read_csv("train.csv")
df.head()


id  carat        cut color clarity  depth  table     x     y     z  price
0   0   1.52    Premium     F     VS2   62.2   58.0  7.27  7.33  4.55  13619
1   1   2.03  Very Good     J     SI2   62.0   58.0  8.06  8.12  5.05  13387
2   2   0.70      Ideal     G     VS1   61.2   57.0  5.69  5.73  3.50   2772
3   3   0.32      Ideal     G     VS1   61.6   56.0  4.38  4.41  2.71    666
4   4   1.70    Premium     G     VS2   62.6   59.0  7.65  7.61  4.77  14453

In [24]:
df=df.drop(labels=['id'],axis=1)


In [25]:

## Independent and dependent features
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]


In [26]:
Y

price
0     13619
1     13387
2      2772
3       666
4     14453
...     ...
1976   6956
1977    499
1978   2085
1979  15619
1980   2705

[1981 rows x 1 columns]

In [27]:

# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns


In [28]:
#Define the custom ranking for each ordinal variable
cut_categories=["Fair","Good","Very Good","Premium","Ideal"]
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [29]:
from sklearn.impute import SimpleImputer  # Handling Missing Values


In [31]:
from sklearn.impute import SimpleImputer  # Handling Missing Values
from sklearn.preprocessing import StandardScaler # Handling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Odinal Encoding
##PIPELINES
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [36]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='median')),
     ('scaler',StandardScaler())
]
)

# Categorical Pipeline

cat_pipeline=Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())

    ]
)

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [37]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [38]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [39]:
# Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [40]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [41]:
regression.coef_

array([[ 6752.6028898 ,   326.39107233,   -17.20964793,   580.27201097,
         1794.95450865, -4893.47888622,   106.51953431,  -379.4668652 ,
          640.17342844]])

In [42]:
regression.intercept_

array([3981.32106782])

In [43]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


In [48]:
## Train multiple models
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    
    #make prediction

    y_pred=model.predict(X_test)
    mae,rmse,r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')
    

LinearRegression
Model Training Performance
RMSE: 1010.579057721137
MAE: 680.3760890879774
R2 score 93.54510245247228


Lasso
Model Training Performance
RMSE: 1014.6386977291157
MAE: 682.3085220935347
R2 score 93.49313780140166


Ridge
Model Training Performance
RMSE: 1012.8195788519408
MAE: 685.0013181052342
R2 score 93.51644884794445


Elasticnet
Model Training Performance
RMSE: 1493.835115099468
MAE: 1073.1534497520604
R2 score 85.89561697273916




c:\HousePricePrediction\.venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.432e+07, tolerance: 2.400e+06
  model = cd_fast.enet_coordinate_descent(


In [49]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']